In [10]:
import pandas as pd
import cartoframes
from cartoframes.auth import set_default_credentials, Credentials
from cartoframes import read_carto
from cartoframes import to_carto
from cartoframes.viz import * 
from cartoframes.data.services import Geocoding
from cartoframes.data.observatory import Dataset, Catalog, Geography, Variable, Enrichment


In [33]:
from cartoframes.auth import set_default_credentials, Credentials
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

carto_username = 'upf28'
carto_API = '0097a629e1a587bd3b71612d65f44f62f1bf0197'

set_default_credentials(username=carto_username, api_key=carto_API)

sns.set()

In [34]:
cities = pd.read_csv('./data/500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2019_release.csv')
cities.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Population2010,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,PAPTEST_Crude95CI,PHLTH_CrudePrev,PHLTH_Crude95CI,SLEEP_CrudePrev,SLEEP_Crude95CI,STROKE_CrudePrev,STROKE_Crude95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,Geolocation
0,AL,Birmingham,107000,1073000100,0107000-01073000100,3042,24.4,"(21.2, 27.8)",31.2,"(30.1, 32.2)",...,"(83.4, 86.6)",20.5,"(18.9, 22.2)",45.5,"(44.3, 46.5)",5.5,"( 5.0, 5.9)",28.9,"(23.7, 35.1)","(33.57943283260, -86.7228323926)"
1,AL,Birmingham,107000,1073000300,0107000-01073000300,2735,30.4,"(27.1, 33.7)",30.5,"(29.3, 31.7)",...,"(81.9, 85.6)",22.2,"(20.4, 24.0)",44.5,"(43.2, 45.7)",6.1,"( 5.5, 6.8)",32.9,"(25.5, 40.2)","(33.54282086860, -86.7524339780)"
2,AL,Birmingham,107000,1073000400,0107000-01073000400,3338,24.7,"(21.1, 28.5)",31.5,"(30.2, 32.9)",...,"(82.3, 86.8)",19.4,"(17.5, 21.4)",45.7,"(44.2, 47.0)",5.6,"( 5.0, 6.2)",28.6,"(22.0, 35.1)","(33.56324496330, -86.7640474064)"
3,AL,Birmingham,107000,1073000500,0107000-01073000500,2864,25.2,"(21.6, 29.0)",35.0,"(33.5, 36.4)",...,"(82.1, 86.5)",21.7,"(19.7, 23.9)",46.1,"(44.5, 47.4)",7.3,"( 6.5, 8.2)",34.0,"(24.4, 42.9)","(33.54424045940, -86.7749130719)"
4,AL,Birmingham,107000,1073000700,0107000-01073000700,2577,29.0,"(24.5, 33.8)",36.3,"(34.9, 37.7)",...,"(82.2, 87.1)",25.5,"(23.1, 28.0)",48.3,"(46.6, 49.7)",9.0,"( 8.1, 10.0)",43.2,"(33.2, 53.2)","(33.55254061390, -86.8016893706)"


In [35]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27210 entries, 0 to 27209
Data columns (total 63 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   StateAbbr               27210 non-null  object 
 1   PlaceName               27210 non-null  object 
 2   PlaceFIPS               27210 non-null  int64  
 3   TractFIPS               27210 non-null  int64  
 4   Place_TractID           27210 non-null  object 
 5   Population2010          27210 non-null  int64  
 6   ACCESS2_CrudePrev       27209 non-null  float64
 7   ACCESS2_Crude95CI       27209 non-null  object 
 8   ARTHRITIS_CrudePrev     27210 non-null  float64
 9   ARTHRITIS_Crude95CI     27210 non-null  object 
 10  BINGE_CrudePrev         27210 non-null  float64
 11  BINGE_Crude95CI         27210 non-null  object 
 12  BPHIGH_CrudePrev        27210 non-null  float64
 13  BPHIGH_Crude95CI        27210 non-null  object 
 14  BPMED_CrudePrev         27210 non-null

In [25]:
cities.Geolocation

0        (33.57943283260, -86.7228323926)
1        (33.54282086860, -86.7524339780)
2        (33.56324496330, -86.7640474064)
3        (33.54424045940, -86.7749130719)
4        (33.55254061390, -86.8016893706)
                       ...               
27205    (41.15850561440, -104.777631834)
27206    (41.17177648050, -104.788212134)
27207    (41.16037417080, -104.756560525)
27208    (41.15063392830, -104.755675636)
27209    (41.13361679520, -104.719053785)
Name: Geolocation, Length: 27210, dtype: object

In [36]:
split_data = cities.Geolocation.str.strip(')').str.strip('(').str.split(', ')
cities['lat'] = split_data.apply(lambda x: x[0])
cities['long'] = split_data.apply(lambda x: x[1])

In [37]:
cities.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Population2010,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,PHLTH_Crude95CI,SLEEP_CrudePrev,SLEEP_Crude95CI,STROKE_CrudePrev,STROKE_Crude95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,Geolocation,lat,long
0,AL,Birmingham,107000,1073000100,0107000-01073000100,3042,24.4,"(21.2, 27.8)",31.2,"(30.1, 32.2)",...,"(18.9, 22.2)",45.5,"(44.3, 46.5)",5.5,"( 5.0, 5.9)",28.9,"(23.7, 35.1)","(33.57943283260, -86.7228323926)",33.57943283260,-86.7228323926
1,AL,Birmingham,107000,1073000300,0107000-01073000300,2735,30.4,"(27.1, 33.7)",30.5,"(29.3, 31.7)",...,"(20.4, 24.0)",44.5,"(43.2, 45.7)",6.1,"( 5.5, 6.8)",32.9,"(25.5, 40.2)","(33.54282086860, -86.7524339780)",33.54282086860,-86.7524339780
2,AL,Birmingham,107000,1073000400,0107000-01073000400,3338,24.7,"(21.1, 28.5)",31.5,"(30.2, 32.9)",...,"(17.5, 21.4)",45.7,"(44.2, 47.0)",5.6,"( 5.0, 6.2)",28.6,"(22.0, 35.1)","(33.56324496330, -86.7640474064)",33.56324496330,-86.7640474064
3,AL,Birmingham,107000,1073000500,0107000-01073000500,2864,25.2,"(21.6, 29.0)",35.0,"(33.5, 36.4)",...,"(19.7, 23.9)",46.1,"(44.5, 47.4)",7.3,"( 6.5, 8.2)",34.0,"(24.4, 42.9)","(33.54424045940, -86.7749130719)",33.54424045940,-86.7749130719
4,AL,Birmingham,107000,1073000700,0107000-01073000700,2577,29.0,"(24.5, 33.8)",36.3,"(34.9, 37.7)",...,"(23.1, 28.0)",48.3,"(46.6, 49.7)",9.0,"( 8.1, 10.0)",43.2,"(33.2, 53.2)","(33.55254061390, -86.8016893706)",33.55254061390,-86.8016893706


In [16]:
import geopandas as gpd
#stores = gpd.read_file('./data/500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2019_release.csv')
#stores

In [39]:
cities_gdf = gpd.GeoDataFrame(
    cities, geometry=gpd.points_from_xy(cities.lat, cities.long))

In [43]:
cities_gdf.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Population2010,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,SLEEP_CrudePrev,SLEEP_Crude95CI,STROKE_CrudePrev,STROKE_Crude95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,Geolocation,lat,long,geometry
0,AL,Birmingham,107000,1073000100,0107000-01073000100,3042,24.4,"(21.2, 27.8)",31.2,"(30.1, 32.2)",...,45.5,"(44.3, 46.5)",5.5,"( 5.0, 5.9)",28.9,"(23.7, 35.1)","(33.57943283260, -86.7228323926)",33.57943283260,-86.7228323926,POINT (33.57943 -86.72283)
1,AL,Birmingham,107000,1073000300,0107000-01073000300,2735,30.4,"(27.1, 33.7)",30.5,"(29.3, 31.7)",...,44.5,"(43.2, 45.7)",6.1,"( 5.5, 6.8)",32.9,"(25.5, 40.2)","(33.54282086860, -86.7524339780)",33.54282086860,-86.7524339780,POINT (33.54282 -86.75243)
2,AL,Birmingham,107000,1073000400,0107000-01073000400,3338,24.7,"(21.1, 28.5)",31.5,"(30.2, 32.9)",...,45.7,"(44.2, 47.0)",5.6,"( 5.0, 6.2)",28.6,"(22.0, 35.1)","(33.56324496330, -86.7640474064)",33.56324496330,-86.7640474064,POINT (33.56324 -86.76405)
3,AL,Birmingham,107000,1073000500,0107000-01073000500,2864,25.2,"(21.6, 29.0)",35.0,"(33.5, 36.4)",...,46.1,"(44.5, 47.4)",7.3,"( 6.5, 8.2)",34.0,"(24.4, 42.9)","(33.54424045940, -86.7749130719)",33.54424045940,-86.7749130719,POINT (33.54424 -86.77491)
4,AL,Birmingham,107000,1073000700,0107000-01073000700,2577,29.0,"(24.5, 33.8)",36.3,"(34.9, 37.7)",...,48.3,"(46.6, 49.7)",9.0,"( 8.1, 10.0)",43.2,"(33.2, 53.2)","(33.55254061390, -86.8016893706)",33.55254061390,-86.8016893706,POINT (33.55254 -86.80169)


In [44]:
cities_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 27210 entries, 0 to 27209
Data columns (total 66 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   StateAbbr               27210 non-null  object  
 1   PlaceName               27210 non-null  object  
 2   PlaceFIPS               27210 non-null  int64   
 3   TractFIPS               27210 non-null  int64   
 4   Place_TractID           27210 non-null  object  
 5   Population2010          27210 non-null  int64   
 6   ACCESS2_CrudePrev       27209 non-null  float64 
 7   ACCESS2_Crude95CI       27209 non-null  object  
 8   ARTHRITIS_CrudePrev     27210 non-null  float64 
 9   ARTHRITIS_Crude95CI     27210 non-null  object  
 10  BINGE_CrudePrev         27210 non-null  float64 
 11  BINGE_Crude95CI         27210 non-null  object  
 12  BPHIGH_CrudePrev        27210 non-null  float64 
 13  BPHIGH_Crude95CI        27210 non-null  object  
 14  BPMED_CrudePre

In [46]:
from cartoframes.viz import Map, Layer

Map([
    Layer(cities_gdf)
])

In [48]:
ranges = [400000, 700000, 1000000, 1300000, 1600000]

Map(
    Layer(cities_gdf, geom_col='geometry'),show_info = True,
    viewport={'zoom': 10, 'lat': 40.646891, 'lng': -73.869378},
    size=(920,400)
)